In [155]:
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torch

In [156]:
data = pd.read_csv('../../데이터/Transaction/transaction_final.csv',index_col=0)

In [157]:
class RNN_Transaction_Dataset(Dataset):
    def __init__(self, data, sequence_length=5):
        data['계약년월'] = pd.to_datetime(data['계약년월'])
        interpolated_data = pd.DataFrame(data.groupby(['시군구', '단지명']).apply(price_per_pyeong_interpolate)['평단가']).reset_index().rename(columns={'level_2': '계약년월'})
        dongs_data = []
        for dong in interpolated_data['시군구'].unique():
            dong_data = []
            for apartment_complex in interpolated_data[interpolated_data['시군구'] == dong]['단지명'].unique():
                filtered_interpolated_data_values = interpolated_data[interpolated_data['단지명'] == apartment_complex]['평단가'].values
                for idx in range(len(filtered_interpolated_data_values) - sequence_length):
                    apartment_complex_x = filtered_interpolated_data_values[idx:idx + sequence_length]
                    apartment_complex_y = filtered_interpolated_data_values[idx + sequence_length:idx + sequence_length + 1]
                    dong_data.append((apartment_complex_x, apartment_complex_y))
            dongs_data.append(dong_data)
        self.dongs_data = dongs_data
        self.len = len(dongs_data)

    # 동 하나 가져오기(따라서 batch_size도 1로 해야 함)
    def __getitem__(self, i):
        dong_data = self.dongs_data[i]
        dong_x, dong_y = [], []
        for apartment_complex_x, apartment_complex_y in dong_data:
            dong_x.append(torch.FloatTensor(apartment_complex_x))
            dong_y.append(torch.FloatTensor(apartment_complex_y))
        return dong_x, dong_y

    # 전체 동 개수
    def __len__(self):
        return self.len

batch_size = 1
train_dataset = RNN_Transaction_Dataset(data)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [ ]:
class ODE_Transaction_Dataset(Dataset):
    def __init__(self, data, sequence_length=5):
        data['계약년월'] = pd.to_datetime(data['계약년월'])
        interpolated_data = pd.DataFrame(data.groupby(['시군구', '단지명']).apply(price_per_pyeong_interpolate)['평단가']).reset_index().rename(columns={'level_2': '계약년월'})
        dongs_data = []
        for dong in interpolated_data['시군구'].unique():
            dong_data = []
            for apartment_complex in interpolated_data[interpolated_data['시군구'] == dong]['단지명'].unique():
                filtered_interpolated_data_values = interpolated_data[interpolated_data['단지명'] == apartment_complex]['평단가'].values
                for idx in range(len(filtered_interpolated_data_values) - sequence_length):
                    apartment_complex_x = filtered_interpolated_data_values[idx:idx + sequence_length]
                    apartment_complex_y = filtered_interpolated_data_values[idx + sequence_length:idx + sequence_length + 1]
                    dong_data.append((apartment_complex_x, apartment_complex_y))
            dongs_data.append(dong_data)
        self.dongs_data = dongs_data
        self.len = len(dongs_data)

    # 동 하나 가져오기(따라서 batch_size도 1로 해야 함)
    def __getitem__(self, i):
        dong_data = self.dongs_data[i]
        dong_x, dong_y = [], []
        for apartment_complex_x, apartment_complex_y in dong_data:
            dong_x.append(torch.FloatTensor(apartment_complex_x))
            dong_y.append(torch.FloatTensor(apartment_complex_y))
        return dong_x, dong_y

    # 전체 동 개수
    def __len__(self):
        return self.len

batch_size = 1
train_dataset = RNN_Transaction_Dataset(data)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [8]:
import torch
from torch.utils.data import Dataset

class Filled_RNN_Dataset(Dataset):
    def __init__(self, data, sequence_length=5):
        interpolated_data = pd.DataFrame(data.groupby(['시군구','단지명']).apply(price_per_pyeong_interpolate)['평단가']).reset_index().rename(columns={'level_2': '계약년월'}).rename(columns={'level_2': '계약년월'})

        # max_complex_len = 0
        # for dong in interpolated_data['시군구'].unique():
        #     dong_unique_complexes_len = len(data[data['시군구']==dong]['단지명'].unique())
        #     if dong_unique_complexes_len > max_complex_len:
        #         max_complex_len = dong_unique_complexes_len

        dongs_X, dongs_Y = [], []

        for i in range(len(pd.to_datetime(pd.date_range(start="2006-01-01", end="2022-12-01", freq='MS')))-sequence_length):
            
        

        # dong_X, dong_Y = [], []
        # for complex in unique_complexes:
        #     filtered_data_values = data[data['단지명']==complex]['평단가'].values
        #     for i in range(len(filtered_data_values) - sequence_length):
        #         X.append(filtered_data_values[i:i+sequence_length])
        #         Y.append(filtered_data_values[i+sequence_length])
        # self.x = torch.FloatTensor(X)
        # self.y = torch.FloatTensor(Y)
        # self.len = len(X)

    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return self.len

sample_dataset = Filled_RNN_Dataset(data)

158
